In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
from torch.hub import load_state_dict_from_url
from torch.utils.data import DataLoader
import torchvision
import keras
from keras.utils import to_categorical

In [ ]:
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
print(data.head)

In [ ]:
data.size
data.shape
data.columns

In [ ]:
features = ['label']
num_data = data['label'].nunique(1)
print(num_data)

In [ ]:
data.index

In [ ]:
data.iloc[0]

This shows that for number one we have the above pixel values.
We will use this to get how many numbers do we have in the data, and train them in batches.

We will use VGG convolutional network for our classification problem.

In [ ]:
# model building
_all_ = ['VGG', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn',
    'vgg19_bn', 'vgg19']


In [ ]:
model_urls = {
    'vgg11': 'https://download.pytorch.org/models/vgg11-bbd30ac9.pth',
    'vgg13': 'https://download.pytorch.org/models/vgg13-c768596a.pth',
    'vgg16': 'https://download.pytorch.org/models/vgg16-397923af.pth',
    'vgg19': 'https://download.pytorch.org/models/vgg19-dcbb9e9d.pth',
    'vgg11_bn': 'https://download.pytorch.org/models/vgg11_bn-6002323d.pth',
    'vgg13_bn': 'https://download.pytorch.org/models/vgg13_bn-abd245e5.pth',
    'vgg16_bn': 'https://download.pytorch.org/models/vgg16_bn-6c64b313.pth',
    'vgg19_bn': 'https://download.pytorch.org/models/vgg19_bn-c79401a0.pth'
}

In [ ]:
# building the model
class VGG(nn.Module):
  def __init__(self, features, num_classes = 1000, init_weight = True):
    super(VGG, self).__init__()
    self.features = features
    self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
    self.classifier = nn.Sequential(
        nn.Linear(512*7*7, 4096),
        nn.ReLU(True),
        nn.Dropout(),
        nn.Linear(4096, 4096),
        nn.ReLU(True),
        nn.Dropout(),
        nn.Linear(4096, num_classes)
    )
    if init_weight:
      self.initialize_weights()

  def forward(self, X):
    X = self.features(X)
    X = self.avgpool(X)
    X = torch.flatten(X)
    X = self.classifier(X)
    return X

  def initialize_weights(self):
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode = 'fan_out', nonlinearity = 'relu')
        if m.bias is not None:
          nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 0)
        nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, 0, 0.01)
        nn.init.constant_(m.bias, 0)

In [ ]:
def make_layers(cfg, batch_norm = False):
  layers = []
  in_channels = 3
  for i in cfg:
    if i == 'M':
      layers += [nn.MaxPool2d(kernel_size = 2, stride = 2)]
    else:
      conv2d = nn.Conv2d(in_channels, i, kernel_size = 3, padding = 1)
      if batch_norm:
        layers += [conv2d, nn.BatchNorm2d(i), nn.ReLU(True)]
      else:
        layers += [conv2d, nn.ReLU(True)]
      in_channel = i
  return nn.Sequential(*layers)

In [ ]:
cfgs = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

In [ ]:
def _vgg(arch, cfg, batch_norm, pretrained, progress, **kwargs):
  if pretrained:
    kwargs['init_weights'] = False
  model = VGG(make_layers(cfgs[cfg], batch_norm = batch_norm), **kwargs)
  if pretrained:
    state_dict = load_state_dict_from_url(model_urls[arch], progress = progress)
    model = model.load_state_dict(state_dict)
  return model

In [ ]:
def vgg19(pretrained=False, progress=True, **kwargs):
    r"""VGG 19-layer model (configuration "E")
    `"Very Deep Convolutional Networks For Large-Scale Image Recognition" <https://arxiv.org/pdf/1409.1556.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _vgg('vgg19', 'E', False, pretrained, progress, **kwargs)

In [ ]:
y_train = data['label']
x_train = data.drop(['label'], axis = 1)
x_train = x_train
x_train = np.array(x_train.values)
x_train = x_train.reshape(-1, 28 , 28, 1)
y_train = to_categorical(y_train, num_classes = 10)

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(64, kernel_size = 3, strides = 1, padding = 'same',
                                data_format = 'channels_last',
                                activation = 'relu'))
model.add(keras.layers.Conv2D(64, kernel_size = 3, strides = 1, padding = 'same',
                                
                                activation = 'relu'))

model.add(keras.layers.MaxPool2D(pool_size = 2, strides = 2))



model.add(keras.layers.MaxPool2D(pool_size = 2, strides = 2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation = 'relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(10, activation = 'softmax'))

In [ ]:
x_train = x_train.reshape(-1, 784)
x_train = x_train/255
x_train = x_train.reshape(-1, 28, 28, 1)

In [ ]:

model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics = ['accuracy'])

In [ ]:
model.build(input_shape = (None, 28, 28, 1))
model.summary()

In [ ]:
model.fit(x_train, y_train, epochs = 10, batch_size = 64)

In [ ]:
test_data = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
test_data.head

In [ ]:
test_data = np.array(test_data)
test_data = test_data.reshape(-1, 28, 28, 1)
prediction = model.predict(test_data, batch_size = 64)

In [ ]:
predict_labels = np.argmax(prediction, axis = 1)
predict_labels[:5]

In [ ]:
out = np.column_stack((range(1,predict_labels.shape[0]+1), predict_labels))
np.savetxt('submission.csv', out, header = "ImageId,Label",comments = "", fmt = '%d,%d')